In [1]:
import os, sys, pickle
from result import Ok, Err, Result


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path

from src.socotra_loader.requests import *
from src.entities import *
from src import logger
from src.socotra_loader.uploading_context import PolicyCreationContext



In [2]:
models_list = pickle.load(open("models.pkl", "rb"))
models_dict = models_list[1]


from src.socotra_loader.authorization_manager import SocotraAuthorizationManager

session_manager = SocotraAuthorizationManager().__enter__()
session_manager

## Context Base Approach

In [3]:
# ph_locator = None
# policy_locator = None


request = CreatePolicyHolder(models_dict["PolicyHolder"])
response = session_manager.request(request)
assert response.response.status_code == 200
ph_locator = response.response.json()['locator']
ph_locator

'1d50a97b-dc75-4b46-ab92-98c9f66e81f6'

## Contextual Loading V2
__Input__: List of PolicyHolderIds \
__Output__: ContextObject

- Iterate PolicyHolderID
    - Create PolicyHolder
    - Iterate Throuhg PolicyIDs
        - __Create Policy Context__
            - Create Policy
                - CreateAUX
                - IssuePolicy
                - FetchInvoices
            - Iterate Invoices
                - PayInvoices
            - Create Endorsement
                - Issue Endorsement
            - Create Renewal
                - Issue Renewal


In [4]:
models_dict["Endorsement"].dict()

{'retrieved_from_brighcore': None,
 'uploaded_to_socotra': None,
 'endorsementName': 'endorsement.with.repricing',
 'state': None,
 'startTimestamp': None,
 'newPolicyEndTimestamp': None,
 'fieldValues': {'rater_source': 'AA0000',
  'rater_id': 'AA0000',
  'custom_policy_number': 'AA3331',
  'auto_renewal_status': 'renew'},
 'addFieldGroups': [{'fieldName': 'non_primary_policy_holders',
   'fieldValues': {'non_primary_name': 'Rosemarie Lehner',
    'non_primary_email': 'Jaquelin31@yahoo.com',
    'non_primary_phone': '938-361-7083',
    'non_primary_dob': datetime.date(2000, 1, 1),
    'non_primary_custom_member_id': 'AA3331',
    'non_primary_relationship': None}}],
 'updateFieldGroups': [{'fieldName': 'building_details',
   'fieldValues': {'address_line_1': '2423 Dickens Squares',
    'address_line_2': '769 Mosciski Canyon',
    'address_city': 'South Derickmouth',
    'address_state': 'NY',
    'address_zip': '10004',
    'address_county': 'New York County',
    'building_id': 'w',


In [5]:
# %%timeit -n 1 -r 1
with PolicyCreationContext(ph_locator, session_manager) as ctx: # for now PHId -> Future PolicyUUID from storage
    ctx.create_policy(models_dict["PolicyProtect"])
    ctx.submit_request("AuxData", models_dict["AuxData"])
    ctx.submit_request("Endorsement", models_dict["Endorsement"])
    ctx.submit_request("Renewal", models_dict["Renewal"])
            

2023-08-07 13:37:05.861 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy - AA3332', 'response': 200}


> /home/lytvyn/projects/jetty/jetty-migration/src/socotra_loader/uploading_context.py(31)submit_request()
     30                 ipdb.set_trace()
---> 31                 self.create_aux(entity)
     32                 self.issue_policy()

ipdb> c


2023-08-07 13:37:08.229 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/auxData/100064120 - 100064120', 'response': 200}
2023-08-07 13:37:10.857 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy/100064120/issue - 100064120', 'response': 200}
2023-08-07 13:37:11.479 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy/100064120/checkInvoices - 100064120', 'response': 200}
2023-08-07 13:37:12.140 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/invoice/ec521edd-2559-4e0c-8241-f43b354d7fc7/pay - ec521edd-2559-4e0c-8241-f43b354d7fc7', 'response': 200}
2023-08-07 13:37:13.006 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policies/100064120/endorsements - 100064120', 'response': 201}
2023-08-07 13:37:15.524 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/endorsement

Exiting Context


In [6]:
%%timeit -n 1 -r 1
# Iterate PH ID
    # Create PolicyHolder
    # Iterate PolicyIDs
# For each Policy create context
with PolicyCreationContext(ph_locator, session_manager) as ctx: # for now PHId -> Future PolicyUUID from storage
# Create Policy
    # Create Request
    ctx.create_policy(models_dict["PolicyProtect"])
    ctx.create_aux(models_dict["AuxData"])
    ctx.issue_policy()
    ctx.fetch_invoices()
    ctx.pay_invoices()
    ctx.create_endorsement(models_dict["Endorsement"])
    ctx.issue_endorsement()
    ctx.create_renewal(models_dict["Renewal"])
    ctx.issue_renewal()

2023-08-07 13:37:19.559 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy - AA3332', 'response': 200}
2023-08-07 13:37:20.158 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/auxData/100064154 - 100064154', 'response': 200}
2023-08-07 13:37:22.434 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy/100064154/issue - 100064154', 'response': 200}
2023-08-07 13:37:23.056 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policy/100064154/checkInvoices - 100064154', 'response': 200}
2023-08-07 13:37:23.716 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/invoice/99946fdf-0115-41f2-b8aa-b822e615226f/pay - 99946fdf-0115-41f2-b8aa-b822e615226f', 'response': 200}
2023-08-07 13:37:24.577 | SUCCESS  | src.socotra_loader.uploading_context:response_hooks:102 - {'request': '/policies/100064154/endorsements - 10006

Exiting Context
11.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
